# 参考mnist_conv-keras实现针对cifar10的卷积模型

## 准备数据

In [12]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def cifar10_dataset():
    (x, y), (x_test, y_test) = datasets.cifar10.load_data()
    
    


## 建立模型

## 编译， fit以及evaluate